# Dependencies

In [ ]:
%pip install tensorflow
%pip install keras
%pip install imutils
%pip install numpy
%pip install opencv-python
%pip install matplotlib
%pip install scipy
%pip install kaggle

# Preprocessing Data

In [6]:
import os 
import numpy as np
import tensorflow.keras.preprocessing.image as Image

from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split


categories = ["with_mask", "without_mask"]
data = []
labels = []
cwd = os.path.join(os.getcwd(), "./dataset")

# Load images & convert them to numeric values
for cat in categories:
    path = os.path.join(cwd, cat)
    for img_name in os.listdir(path):
        img_path = os.path.join(path, img_name)
        img = Image.load_img(img_path, target_size=(224, 224))
        img = Image.img_to_array(img)
        img = preprocess_input(img)
        
        data.append(img)
        labels.append(cat)


# Convert labels to binary values
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

data = np.array(data, dtype="float32")
labels = np.array(labels)

# Split data into training and test sets
(trainX,
 testX,
 trainY,
 testY) = train_test_split(data, labels, test_size=.2, stratify=labels)



# Train model

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers
from tensorflow.keras.models import Model

LEARNING_RATE = 1e-2
EPOCHS = 5
BBATCH_SIZE = 32

# Generate variations of images to extend the dataset
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

# Build base and head models
base_model = MobileNetV2(weights="imagenet",
                         include_top=False,
                         input_tensor=layers.Input(shape=(224, 224, 3)))

head_model = base_model.output
head_model = layers.AveragePooling2D(pool_size=(7, 7))(head_model)
head_model = layers.Flatten(name="flatten")(head_model)
head_model = layers.Dense(128, activation="relu")(head_model)
head_model = layers.Dropout(0.5)(head_model)
head_model = layers.Dense(2, activation="softmax")(head_model)

model = Model(inputs=base_model.input, outputs=head_model)

# Freeze base model layers from being updated during the first training
for layer in base_model.layers:
    layer.trainable = False
    
# Compile the model
# opt = Adam(lr=LEARNING_RATE, decay=LEARNING_RATE / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train network's head
head = model.fit(
	aug.flow(trainX, trainY, batch_size=BBATCH_SIZE),
	steps_per_epoch=len(trainX) // BBATCH_SIZE,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BBATCH_SIZE,
	epochs=EPOCHS)

# Evaluate network
pred_indices = model.predict(testX, batch_size=BBATCH_SIZE)
pred_indices = np.argmax(pred_indices, axis=1)
model.save("face_mask_detector.model.h5")

/var/folders/x3/knkkttzs2pq6kykbhbxr4wjr0000gn/T/ipykernel_55532/1322467963.py:21: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights="imagenet",


Epoch 1/5


/Users/amir/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


149/149 ━━━━━━━━━━━━━━━━━━━━ 58s 366ms/step - accuracy: 0.8080 - loss: 0.4442 - val_accuracy: 0.8464 - val_loss: 0.3504
Epoch 2/5
149/149 ━━━━━━━━━━━━━━━━━━━━ 0s 554us/step - accuracy: 0.9062 - loss: 0.2595 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/5


2024-04-14 16:45:27.943973: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Users/amir/anaconda3/lib/python3.11/contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
2024-04-14 16:45:27.966240: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


149/149 ━━━━━━━━━━━━━━━━━━━━ 59s 387ms/step - accuracy: 0.9005 - loss: 0.2730 - val_accuracy: 0.8823 - val_loss: 0.2898
Epoch 4/5
149/149 ━━━━━━━━━━━━━━━━━━━━ 0s 504us/step - accuracy: 0.8750 - loss: 0.2965 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/5


2024-04-14 16:46:27.386587: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-04-14 16:46:27.401620: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


149/149 ━━━━━━━━━━━━━━━━━━━━ 59s 388ms/step - accuracy: 0.8955 - loss: 0.2673 - val_accuracy: 0.8973 - val_loss: 0.2393
38/38 ━━━━━━━━━━━━━━━━━━━━ 11s 280ms/step


# Test

In [5]:
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.applications import MobileNetV2
# from tensorflow.keras.layers import AveragePooling2D
# from tensorflow.keras.layers import Dropout
# from tensorflow.keras.layers import Flatten
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.layers import Input
# from tensorflow.keras.models import Model
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
# from tensorflow.keras.preprocessing.image import img_to_array
# from tensorflow.keras.preprocessing.image import load_img
# from tensorflow.keras.utils import to_categorical
# from sklearn.preprocessing import LabelBinarizer
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report
# from imutils import paths
# import matplotlib.pyplot as plt
# import numpy as np
# import os

# INIT_LR = 1e-4
# EPOCHS = 20
# BS = 32

# # construct the training image generator for data augmentation
# aug = ImageDataGenerator(
# 	rotation_range=20,
# 	zoom_range=0.15,
# 	width_shift_range=0.2,
# 	height_shift_range=0.2,
# 	shear_range=0.15,
# 	horizontal_flip=True,
# 	fill_mode="nearest")

# # load the MobileNetV2 network, ensuring the head FC layer sets are
# # left off
# baseModel = MobileNetV2(weights="imagenet", include_top=False,
# 	input_tensor=Input(shape=(224, 224, 3)))

# # construct the head of the model that will be placed on top of the
# # the base model
# headModel = baseModel.output
# headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
# headModel = Flatten(name="flatten")(headModel)
# headModel = Dense(128, activation="relu")(headModel)
# headModel = Dropout(0.5)(headModel)
# headModel = Dense(2, activation="softmax")(headModel)

# # place the head FC model on top of the base model (this will become
# # the actual model we will train)
# model = Model(inputs=baseModel.input, outputs=headModel)

# # loop over all layers in the base model and freeze them so they will
# # *not* be updated during the first training process
# for layer in baseModel.layers:
# 	layer.trainable = False

# # compile our model
# print("[INFO] compiling model...")
# opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
# model.compile(loss="binary_crossentropy", optimizer=opt,
# 	metrics=["accuracy"])

# # train the head of the network
# print("[INFO] training head...")
# H = model.fit(
# 	aug.flow(trainX, trainY, batch_size=BS),
# 	steps_per_epoch=len(trainX) // BS,
# 	validation_data=(testX, testY),
# 	validation_steps=len(testX) // BS,
# 	epochs=EPOCHS)

# # make predictions on the testing set
# print("[INFO] evaluating network...")
# predIdxs = model.predict(testX, batch_size=BS)

# # for each image in the testing set we need to find the index of the
# # label with corresponding largest predicted probability
# predIdxs = np.argmax(predIdxs, axis=1)

# # show a nicely formatted classification report
# print(classification_report(testY.argmax(axis=1), predIdxs,
# 	target_names=lb.classes_))

# # serialize the model to disk
# print("[INFO] saving mask detector model...")
# model.save("mask_detector.model", save_format="h5")


/var/folders/x3/knkkttzs2pq6kykbhbxr4wjr0000gn/T/ipykernel_55532/2744930519.py:38: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  baseModel = MobileNetV2(weights="imagenet", include_top=False,


[INFO] compiling model...


/Users/amir/anaconda3/lib/python3.11/site-packages/keras/src/optimizers/base_optimizer.py:34: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


ValueError: Argument(s) not recognized: {'lr': 0.0001}